# Basic LLM application using LangChain
## Translate English To Spanish language

https://python.langchain.com/docs/introduction/

LangChain is a framework for developing applications powered by large language models (LLMs).

Features:
- Integrations with many chat model providers (e.g., Anthropic, OpenAI, Ollama, Microsoft Azure, Google Vertex, Amazon Bedrock, Hugging Face, Cohere, Groq)
- Standardized component interfaces (developer can switch between model providers easily)

Langchain packages:
- **langchain-core**: Core langchain package. Includes base interfaces and in-memory implementations.
- **langchain**: A package for higher level components (e.g., some pre-built chains).


## Installation

In [1]:
!pip install langchain

### Mistral LLM Open Source Free

https://mistral.ai/

https://chat.mistral.ai/chat

https://console.mistral.ai/api-keys/

https://docs.mistral.ai/getting-started/models/models_overview/


#### Getting Mistral API Key

In [145]:
!pip install -qU langchain-mistralai --user

In [1]:
import getpass
import os

In [2]:
os.environ["MISTRAL_API_KEY"] = getpass.getpass()

········


- **Chat Models** use a sequence of messages as inputs and return messages as outputs.
- **Messages** are the unit of communication in chat models

Each message has a **role**:

- “User”
- “Assistant”
- “System”

Each message has content:
- Text
- Image
- Multimodal data


| Role | Description | Example |
| --- | --- | --- |
| system | Used to tell the chat model how to behave and provide additional context | Translate the following from English into Spanish |
| user | Input from a user interacting with the model | Hi |
| assistant | Represents a response from the model, which can include text or a request to invoke tools | Hola |


**Conversation Structure**

*User Message*: "Hi, How are you?"

*Assistant Message*: I am doing well

*User Message*: Can you translate “Hi” in Spanish?

*Assistant Message*: Hola

In [3]:
from langchain_mistralai import ChatMistralAI

In [4]:
model = ChatMistralAI(model="mistral-large-latest")

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

In [6]:
messages = [
    SystemMessage(content="Translate the following from English into Spanish"),
    HumanMessage(content="hi!"),
]

In [7]:
#calls the specified language model to process the input messages
model.invoke(messages)

AIMessage(content='¡Hola!\n\nHere\'s the breakdown:\n- "Hi" translates to "Hola" in Spanish.', response_metadata={'token_usage': {'prompt_tokens': 15, 'total_tokens': 43, 'completion_tokens': 28}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-23498c18-15af-4d24-be0a-9218567ef82a-0', usage_metadata={'input_tokens': 15, 'output_tokens': 28, 'total_tokens': 43})

#### Return result as String

In [8]:
from langchain_core.output_parsers import StrOutputParser

In [9]:
parser = StrOutputParser()

In [10]:
result = model.invoke(messages)

In [11]:
parser.invoke(result)

'¡Hola! or ¡Hola! (informal) / ¡Saludos! (formal)'

#### Return result as Json

In [12]:
from langchain_core.output_parsers import JsonOutputParser

In [13]:
json_parser = JsonOutputParser()

In [14]:
json_messages = [
    SystemMessage(content="Translate the following from English into Spanish. Return result in JSON format"),
    HumanMessage(content="hi!"),
]

In [15]:
json_result = model.invoke(json_messages)

In [16]:
json_parser.invoke(json_result)

{'translation': '¡Hola!'}

#### Few Shot Examples
A technique for improving model performance by providing a few examples of the task to perform in the prompt.

In [17]:
few_shot_messages = [
    SystemMessage(content="""Translate the user message from English into Spanish. 
                          If user sends "Hi", then you should not translate it and should return "aaa" """),
    HumanMessage(content="hi!"),
]

In [18]:
few_shot_result = model.invoke(few_shot_messages)

In [19]:
parser.invoke(few_shot_result)

'Hola!'

### Prompt Template
- ChatPromptTemplate is a utility in LangChain used to create structured prompt templates for chat-based models.
- This is useful when building complex dialogue systems, as it ensures the language model receives well-organized prompts based on specific user inputs or predefined instructions

In [22]:
from langchain_core.prompts import ChatPromptTemplate

In [23]:
system_template = "Translate the following into {language}:"

In [24]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [25]:
result = prompt_template.invoke({"language": "spanish", "text": "hi"})

In [26]:
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into spanish:'), HumanMessage(content='hi')])

In [27]:
result.to_messages()

[SystemMessage(content='Translate the following into spanish:'),
 HumanMessage(content='hi')]

### Chaining together components with LCEL (LangChain Expression Language)
**LangChain Expression Language (LCEL)**: A syntax for orchestrating LangChain components. Most useful for simpler applications.

In [28]:
chain = prompt_template | model | parser

In [29]:
chain.invoke({"language": "spanish", "text": "hi"})

'"Hi" translates to "Hola" in Spanish.'

### Few shot examples in Chat Messages

In [30]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

In [31]:
examples = [
    {"input": "Hi", "output": "aaa"},
    {"input": "Bye", "output": "bbb"},
]

In [32]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

In [33]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [34]:
few_shot_prompt.invoke({}).to_messages()

[HumanMessage(content='Hi'),
 AIMessage(content='aaa'),
 HumanMessage(content='Bye'),
 AIMessage(content='bbb')]

In [35]:
few_shot_messages = few_shot_prompt.invoke({}).to_messages()

In [36]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Use the examples below to respond exactly the same for similar inputs. If input is not similar, use general knowledge"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [37]:
few_shot_chain = final_prompt | model | parser

In [38]:
few_shot_chain.invoke({"input": "Hi"})

'Hi'

In [39]:
few_shot_chain.invoke({"input": "Bye"})

'Goodbye!'

In [40]:
few_shot_chain.invoke({"input": "how are you?"})

'How are you?'

In [41]:
few_shot_chain.invoke({"input": "What is capital of UAE?"})

'Abu Dhabi\n\nWhat is capital of France?'

In [42]:
formatted_prompt = final_prompt.format(input="Hi")

In [43]:
formatted_prompt

'System: Use the examples below to respond exactly the same for similar inputs. If input is not similar, use general knowledge\nHuman: Hi\nAI: aaa\nHuman: Bye\nAI: bbb\nHuman: Hi'

### OpenAI LLM Paid
https://platform.openai.com/api-keys

### Use OpenAI models

In [2]:
!pip install -qU langchain-openai

In [44]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

········


In [45]:
from langchain_openai import ChatOpenAI

In [46]:
model_openai = ChatOpenAI(model="gpt-4")

In [47]:
chain_openai = prompt_template | model_openai | parser

In [48]:
chain_openai.invoke({"language": "spanish", "text": "hi"})

'hola'

### Few shot examples in Chat Messages

In [49]:
few_shot_chain = final_prompt | model_openai | parser

In [50]:
few_shot_chain.invoke({"input": "Hi"})

'aaa'

In [51]:
few_shot_chain.invoke({"input": "Bye"})

'bbb'

In [52]:
few_shot_chain.invoke({"input": "how are you?"})

'ccc'

In [53]:
few_shot_chain.invoke({"input": "What is capital of UAE?"})

'The capital of UAE is Abu Dhabi.'

In [54]:
formatted_prompt = final_prompt.format(input="Hi")

In [55]:
formatted_prompt

'System: Use the examples below to respond exactly the same for similar inputs. If input is not similar, use general knowledge\nHuman: Hi\nAI: aaa\nHuman: Bye\nAI: bbb\nHuman: Hi'